In [283]:
import glob
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
# import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
import time

In [2]:
# path = './hw1_git/11775-hws/videos/*.mp4'
path = './hw1_git/11775-hws/hw1_code/mfcc/*.csv'

filelist = []

for file in glob.glob(path):
    filelist.append(file)

#### To Load MFCC features

In [28]:
train_sample = pd.read_csv('./hw1_git/11775-hws/hw1_code/select.mfcc.csv', header=None, sep=';')
start_time = time.time()
n_clusters = 50
n_init = 5
foo = KMeans(n_clusters = n_clusters, random_state = 0, n_init = n_init, n_jobs = -1).fit(train_sample)
print("===== The time consuming of Kmeans clustering : {} seconds =====".format((time.time() - start_time)))

===== The time consuming of Kmeans clustering : 2595.11231899 seconds =====


In [38]:
import pickle
# save the model to disk
filename = './hw1_git/11775-hws/hw1_code/models/KMeans_k100_iter5.sav'
pickle.dump(foo, open(filename, 'wb'))

In [154]:
# some time later...
# load the model from disk
import pickle
filename = './hw1_git/11775-hws/hw1_code/models/KMeans_k100_iter5.sav'
model = pickle.load(open(filename, 'rb'))
print(model)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=100, n_init=5, n_jobs=-1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)


In [155]:
def get_features(k, model, path_list):
    loaded_model= model
    start_time = time.time()
    features_dict = dict()
    filelist = path_list
    for i in range(len(filelist)):
#     for i in range(10):        
        if i % 1000 == 0: 
            print('{}th step progressing....'.format(i)) 
        else: 
            pass
        data = pd.read_csv(filelist[i], sep = ';', header = None)
        pred_centers = loaded_model.predict(data)
        num_clusters = k
        bow_preds = np.zeros((1, num_clusters))

        for ind in pred_centers:
            bow_preds[0, ind] += 1
        norm_feat = (1.0 * bow_preds)/np.sum(bow_preds)
        features_dict[i] = pd.DataFrame(norm_feat)

    features_total = features_dict[0].copy()
    for i in range(1, len(features_dict)):
        foo = features_dict[i].copy()
        features_total = pd.concat([features_total, foo], axis = 0)
        features_total = features_total.reset_index().drop('index', axis = 1)
        
    print("===== The time consuming of getting features : {} seconds =====".format((time.time() - start_time)))
    return features_total

In [156]:
total_features = get_features(100, model = model, path_list = filelist)

0th step progressing....
1000th step progressing....
2000th step progressing....
===== The time consuming of getting features : 450.531579971 seconds =====


In [267]:
total_features.to_csv('./hw1_git/11775-hws/hw1_code/total_features_k100.csv', index=False)

In [157]:
# Making Video name
video_name_ind = []
for i in range(len(filelist)):
    import re
    match_front = re.search('mfcc/', filelist[i])
    match_end = re.search('.mfcc.csv', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})
#     print(i, filelist[i][match_front.end():match_end.start()])


In [158]:
# Making features columns
k = 100
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

total_data = pd.concat([video_name, total_features], axis = 1)
total_data.columns = column_name

In [159]:
total_data

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99
0,HVC1037,0.005898,0.078637,0.004587,0.000000,0.000000,0.000000,0.134338,0.000000,0.000000,...,0.000655,0.000655,0.000000,0.001311,0.000000,0.079292,0.001966,0.060288,0.000000,0.000000
1,HVC4219,0.000000,0.000000,0.034764,0.000000,0.016445,0.039541,0.000000,0.002781,0.000242,...,0.000000,0.003204,0.000726,0.000000,0.000121,0.002177,0.000846,0.000423,0.000060,0.046010
2,HVC4734,0.000000,0.000000,0.000615,0.000000,0.027730,0.011190,0.000000,0.000061,0.002582,...,0.000000,0.000000,0.000676,0.000000,0.000061,0.000000,0.000000,0.000000,0.000000,0.006333
3,HVC3335,0.001246,0.000000,0.004984,0.006854,0.000000,0.000000,0.000000,0.003115,0.000000,...,0.029907,0.000000,0.000000,0.094081,0.034891,0.000000,0.054829,0.000000,0.004361,0.000000
4,HVC2145,0.000361,0.007291,0.000289,0.028875,0.000000,0.000433,0.000361,0.002815,0.000144,...,0.022378,0.003537,0.000072,0.000794,0.000144,0.008951,0.000433,0.032412,0.004620,0.000217
5,HVC5021,0.000000,0.000000,0.000412,0.000000,0.010472,0.007091,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000330,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009647
6,HVC1136,0.023511,0.006792,0.042842,0.000000,0.000000,0.000000,0.024033,0.000000,0.002090,...,0.000000,0.042320,0.001567,0.000000,0.001567,0.000000,0.025601,0.000522,0.000000,0.000000
7,HVC3599,0.003238,0.026785,0.000126,0.001598,0.000000,0.000000,0.007190,0.000000,0.000000,...,0.002060,0.000126,0.007064,0.001051,0.000547,0.000000,0.004457,0.000378,0.000799,0.000000
8,HVC6015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062258,0.003006,...,0.000000,0.000429,0.024045,0.000429,0.002147,0.062258,0.003435,0.006870,0.001288,0.002147
9,HVC2696,0.000000,0.000000,0.003197,0.000000,0.000320,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000320,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [160]:
train_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/test.video', sep = ' ', header = None)

train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']

In [223]:
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/di

In [224]:
total_mart = total_data.merge(data_lable, how = 'left', on = 'video')

In [225]:
train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

In [226]:
print(train_mart.shape, valid_mart.shape, test_mart.shape)

((816, 109), (390, 109), (1651, 109))


#### Modeling....

In [227]:
train_mart.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_98,feature_99,target,Data,target_p001,target_p002,target_p003,target_p001_10,target_p002_10,target_p003_10
4,HVC2145,0.000361,0.007291,0.000289,0.028875,0.0,0.000433,0.000361,0.002815,0.000144,...,0.004620,0.000217,P003,TRAIN,Other,Other,P003,0,0,1
7,HVC3599,0.003238,0.026785,0.000126,0.001598,0.0,0.000000,0.007190,0.000000,0.000000,...,0.000799,0.000000,NaN,TRAIN,Other,Other,Other,0,0,0
8,HVC6015,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.062258,0.003006,...,0.001288,0.002147,NaN,TRAIN,Other,Other,Other,0,0,0


In [246]:
# svc = SVC(kernel=chi2_kernel, probability=True)
# svc.fit(X_train, y_train)
# y_preds = svc.predict(X_train)
# y_probs = svc.predict_proba(X_train)

# boost = GradientBoostingClassifier(n_estimators=200, random_state=0)
# boost.fit(X_train, y_train)
# y_preds = boost.predict(X_valid)
# y_probs = boost.predict_proba(X_valid)

X_train = train_mart.iloc[:,1:k+1]
y_train = train_mart['target_p001_10']
X_valid = valid_mart.iloc[:,1:k+1]
y_valid = valid_mart['target_p001_10']

adab = AdaBoostClassifier(n_estimators=200, random_state=0, )
adab.fit(X_train, y_train)
y_preds = adab.predict(X_valid)
y_probs = adab.predict_proba(X_valid)
average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])

In [287]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

In [288]:
SVM_results_p001 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of SVM Modeling : 0.546509027481 seconds =====
0.08801807113451755
===== The time consuming of SVM Modeling : 0.533987045288 seconds =====
0.422600108936784
===== The time consuming of SVM Modeling : 0.563760995865 seconds =====
0.2438386451598255


In [264]:
AdaB_results_p001 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of Modeling : 1.34818720818 seconds =====
0.14721823771067066
===== The time consuming of Modeling : 1.34099698067 seconds =====
0.4278725862999068
===== The time consuming of Modeling : 1.33819198608 seconds =====
0.13067477535488464


In [275]:
Boost_results_p001 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of Modeling : 0.770385980606 seconds =====
0.08130972183122778
===== The time consuming of Modeling : 0.778630018234 seconds =====
0.44163047601098665
===== The time consuming of Modeling : 0.78382396698 seconds =====
0.20609862899160264


In [286]:
Xgb_results_p001 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of Modeling : 0.344938993454 seconds =====
0.13364857008278846
===== The time consuming of Modeling : 0.334398984909 seconds =====
0.4966432174204221
===== The time consuming of Modeling : 0.357914924622 seconds =====
0.3304264958580653
